In [ ]:
# -*- coding: utf-8-sig -*-
import pandas as pd 
import numpy as np 
import re
from difflib import SequenceMatcher


pattern = re.compile('[\u0627-\u064a]')

recipes = pd.read_json('test1000.json',encoding="utf-8-sig")

#http://languagelog.ldc.upenn.edu/myl/ldc/morph/buckwalter.html
def cleanString(inp):
    # ، بال وبال ال
    out = " ".join(re.findall(r'[\u0621-\u0652]+', inp))
    out = out.replace(u'،',' ')
    out = out.replace(u'ء',' ')

    out = out.replace(' \u0628 ',' ')#/ و /
    out = out.replace('\u0622','\u0627')#آ
    out = out.replace('\u0623','\u0627')#أ
    out = out.replace('\u0625','\u0627')#إ
    
    out = out.replace('\u0629','\u0647')#ة
    out = out.replace('\u0649','\u064A')#ي

    out = out.replace(' \u0628 ',' ')#/ و /
    out = out.replace(' \u0627\u0628 ',' ')#/ او /
    out = out.replace(u'ال',' ')
    out = out.replace(u'لل',' ')
    out = out.replace('\u0624','')#ؤ
    out = out.replace('\u0648',' ')#و
    #out = out.replace('\u0628','')#ب
    out = out.replace('\u0647','')#ه


    
    
    return out

### extract names from ingredients
ingrdsCorpus = []
for ingrds in recipes["ingrd"]:
    for ingrd in ingrds:
        ingrdTxt = cleanString(ingrd["n"])
        for comp in ingrdTxt.split():
            if pattern.match(comp) != None and len(comp)>2:
                ingrdsCorpus.append(comp)
        del ingrdTxt 
        
del recipes

ingrdsCorpus = (set(ingrdsCorpus))



### extract names from prices
prices = pd.read_csv('data.csv',encoding="utf-8-sig")

ItemCorpus = []
for item in prices["n"]:
    item = cleanString(item)
    for comp in item.split():
        if pattern.match(comp) != None and len(comp)>2:
            ItemCorpus.append(comp)

    del item  
    
del prices            
ItemCorpus = (set(ItemCorpus))

names = []
found = []
for i in ingrdsCorpus:
    if not i in ItemCorpus:
        names.append(i) 
    else:
        found.append(i)

    
names=list(set(names))

for word in names:
    for item in ItemCorpus:
        if SequenceMatcher(None, word, item).ratio()>0.9:
            print(u"{}:{}".format(word,item))
            break

print(found)        
del ItemCorpus, ingrdsCorpus, names, found